In [4]:
import pandas as pd

In [5]:
!dir

 Volume in drive C is Windows
 Volume Serial Number is 1C19-2A23

 Directory of C:\Users\HP\Downloads\Zepto

09-08-2024  07:48    <DIR>          .
08-08-2024  22:37    <DIR>          ..
08-08-2024  22:44    <DIR>          .ipynb_checkpoints
09-08-2024  07:44               230 app.py
09-08-2024  07:41             1,746 appe.py
08-08-2024  21:08        38,134,703 flipkart_com-ecommerce_sample.csv
09-08-2024  07:48             9,997 preprocessing.ipynb
09-08-2024  06:38    <DIR>          venv
               4 File(s)     38,146,676 bytes
               4 Dir(s)   4,828,241,920 bytes free


In [6]:
data = pd.read_csv("flipkart_com-ecommerce_sample.csv")

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20002 entries, 0 to 20001
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   uniq_id                  20000 non-null  object 
 1   crawl_timestamp          20000 non-null  object 
 2   product_url              20000 non-null  object 
 3   product_name             20000 non-null  object 
 4   product_category_tree    20000 non-null  object 
 5   pid                      20000 non-null  object 
 6   retail_price             19922 non-null  float64
 7   discounted_price         19922 non-null  float64
 8   image                    19997 non-null  object 
 9   is_FK_Advantage_product  20000 non-null  object 
 10  description              19998 non-null  object 
 11  product_rating           20000 non-null  object 
 12  overall_rating           20000 non-null  object 
 13  brand                    14136 non-null  object 
 14  product_specifications

In [8]:
data['searchable_text'] = (
    data['product_name'].fillna('') + " " +
    data['description'].fillna('') + " " +
    data['brand'].fillna('') + " " +
    data['product_category_tree'].fillna('')
)

In [9]:
data['searchable_text'].head(),data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20002 entries, 0 to 20001
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   uniq_id                  20000 non-null  object 
 1   crawl_timestamp          20000 non-null  object 
 2   product_url              20000 non-null  object 
 3   product_name             20000 non-null  object 
 4   product_category_tree    20000 non-null  object 
 5   pid                      20000 non-null  object 
 6   retail_price             19922 non-null  float64
 7   discounted_price         19922 non-null  float64
 8   image                    19997 non-null  object 
 9   is_FK_Advantage_product  20000 non-null  object 
 10  description              19998 non-null  object 
 11  product_rating           20000 non-null  object 
 12  overall_rating           20000 non-null  object 
 13  brand                    14136 non-null  object 
 14  product_specifications

(0    Alisha Solid Women's Cycling Shorts Key Featur...
 1    FabHomeDecor Fabric Double Sofa Bed FabHomeDec...
 2    AW Bellies Key Features of AW Bellies Sandals ...
 3    Alisha Solid Women's Cycling Shorts Key Featur...
 4    Sicons All Purpose Arnica Dog Shampoo Specific...
 Name: searchable_text, dtype: object,
 None)

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)

tfidf_matrix = vectorizer.fit_transform(data['searchable_text'])

tfidf_matrix.shape

(20002, 5000)

In [11]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

class RealTimeSearchSystem:
    def __init__(self, tfidf_matrix, data, vectorizer):
        self.tfidf_matrix = tfidf_matrix
        self.data = data
        self.vectorizer = vectorizer

    def search(self, query, top_n=10):
        query_vec = self.vectorizer.transform([query])
        similarities = cosine_similarity(query_vec, self.tfidf_matrix).flatten()
        top_indices = similarities.argsort()[-top_n:][::-1]
        return self.data.iloc[top_indices][['product_name', 'brand', 'product_category_tree', 'description']], similarities[top_indices]


In [12]:
search_system = RealTimeSearchSystem(tfidf_matrix, data, vectorizer)
example_query = "mobile phone"
search_results, similarities = search_system.search(example_query)

search_results["description"], similarities

(4886    Buy Aeoss Bike Phone Support Mobile holder for...
 1702    kasemantra Back Cover for Apple iPad Mini (Mul...
 1869    kasemantra Back Cover for Apple iPad Mini (Mul...
 1756    Kasemantra Back Cover for Apple iPad Mini (Mul...
 1883    kasemantra Back Cover for Apple iPad Mini (Mul...
 1876    kasemantra Back Cover for Apple iPad Mini (Gol...
 1658    Kasemantra Back Cover for Apple iPad Mini (Yel...
 1674    Kasemantra Back Cover for Apple iPad Mini (Bro...
 1711    Kasemantra Back Cover for Apple iPad Mini (Bro...
 1853    kasemantra Back Cover for Apple iPad Mini (Dar...
 Name: description, dtype: object,
 array([0.55002581, 0.474345  , 0.474345  , 0.474345  , 0.474345  ,
        0.47434342, 0.47429974, 0.47408838, 0.47408838, 0.47367577]))

In [13]:
import joblib

In [14]:
joblib.dump(tfidf_matrix, 'vectors.joblib')

['vectors.joblib']

In [15]:
joblib.dump(data, 'data.joblib')

['data.joblib']

In [16]:
print(joblib.__version__)

1.1.1
